In [2]:
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
import pickle
import pandas as pd
import collections
from nltk.corpus import words

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
def load_data(filename='all_songs.csv',
              col='album',
              col_value='In the Aeroplane Over the Sea'):
    
    df    = pd.read_csv('all_songs.csv')
    df    = df.loc[df[col] == col_value]
    songs = df['lyrics'].values
    return songs

def lyric_cleaner(songs):
    lyric_tokens = []
    for song in songs:
        text = song.lower().replace(' n ', ' eol ').replace('[verse ', '[verse')
        text = text.replace("'", '').replace('-', ' ')
        tokens = text.split()
        table = str.maketrans('', '', string.punctuation)
        tokens = [word.translate(table) for word in tokens]
        lyric_tokens.append(tokens)
    
    return lyric_tokens

def join_song_lyrics(lyric_tokens):
    
    joined_songs = []
    
    for song in lyric_tokens:
        lyrics = ' '.join(song)
        joined_songs.append(lyrics)
        
    return joined_songs

def tokenizer(list_of_songs, tokenizer):
    tokenizer = tokenizer
    tokenizer.fit_on_texts(list_of_songs)
    encoded_songs = tokenizer.texts_to_sequences(list_of_songs)
    
    return encoded_songs, tokenizer

def find_longest_song(encoded_songs):
    longest_song_size = 0
    for song in encoded_songs:
        if len(song) > longest_song_size:
            longest_song_size = len(song)
    return longest_song_size

def index_to_word(tokenizer):
    vocab_size = len(tokenizer.word_index) + 1
    index_to_word_dict =  dict([(index, word) for word, index in tokenizer.word_index.items()])
    return vocab_size, index_to_word_dict

def pad_data(encoded_songs, max_length):
    padded_songs = pad_sequences(encoded_songs, maxlen=max_length, padding='post', truncating='post')
    return padded_songs

def seqeuncer(padded_songs, length):
    sequences = []
    output = []

    for song in padded_songs:
        for i in range(length, (len(song))):
            seq = song[i - length: i]
            out = song[i]
            sequences.append(seq)
            output.append(out)
            
    return np.array(sequences), output

def establish_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def gen_seed(seqs):
    seed_text = seqs[np.random.randint(0,len(seqs))]
    while seed_text.all() == np.array([  0, 0,  0, 0, 0, 0, 0, 0, 0, 0]).all():
        seed_text = seqs[np.random.randint(0,len(seqs))]
    return seed_text

def generate_seq(model,
                 tokenizer,
                 seq_length,
                 seqs,
                 n_words=1000):
    
    seed_text = gen_seed()
    start = [reverse_dict[index] for index in seed_text]
    print('start: ', start)
    
    result = []
    
    for _ in range(n_words):
        encoded = pad_sequences([seed_text], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        if yhat == 0:
            pass
        else:
            word = reverse_dict[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-10:]
        result.append(word)
    return ' '.join(result)

In [4]:
sequences[0][0:11]

NameError: name 'sequences' is not defined

In [5]:
X.shape

NameError: name 'X' is not defined

In [6]:
data = load_data(filename='all_songs.csv', col='artist', col_value='The Decemberists')

token_lyrics = lyric_cleaner(data)

In [7]:
words = []
for song in token_lyrics:
    song.append('eos')
    for word in song:
        words.append(word)

word_count = collections.Counter(words)

len_vocab = 2500

most_common = word_count.most_common(n=len_vocab)

vocab = []
for word, count in most_common:
    vocab.append(word)

word_to_index = dict(zip(vocab, range(0, len(vocab))))
# word_to_index['eos'] = 0
word_to_index['unknown'] = len(vocab)

index_to_word = dict([(index, word) for word, index in word_to_index.items()])

In [13]:
with open('the_decemberists_word_to_index', 'wb') as f:
    pickle.dump(word_to_index, file=f)

In [14]:
with open('the_decemberists_index_to_word', 'wb') as f:
    pickle.dump(index_to_word, file=f)

In [4]:
encoded_words = [word_to_index[word] if word in word_to_index else word_to_index['unknown'] for word in words]

length = 25 + 1
sequences = []
for i in range(length, len(encoded_words)):
    seq = encoded_words[i-length:i]
#     line = ' '.join(seq)
    sequences.append(seq)

n_patterns = len(sequences)

sequences = np.array(sequences)

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y)
seq_length = X.shape[1]

In [61]:
model = Sequential()
model.add(Embedding(len_vocab, 50, input_length=25))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(len_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
filepath="model_checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [38]:
model.fit(X, y, batch_size=128, epochs=100, callbacks=callbacks_list)

NameError: name 'model' is not defined

In [88]:
def gen_seed(sequences):
    seed_text = sequences[np.random.randint(0,len(sequences))]
    return seed_text

In [ ]:
seed_text.all() == np.array([  0, 0,  0, 0, 0, 0, 0, 0, 0, 0]).all():

In [84]:
seed_text = sequences[np.random.randint(0,len(sequences))][:25]

In [104]:
seed_text.shape

(25,)

In [136]:
X

array([[139,   0,   2, ...,  11, 396,   1],
       [  0,   2, 396, ..., 396,   1, 300],
       [  2, 396,   1, ...,   1, 300, 950],
       ...,
       [269,   4,   2, ...,  24,  63, 187],
       [  4,   2,   5, ...,  63, 187,  19],
       [  2,   5, 860, ..., 187,  19, 695]])

In [138]:
test = np.expand_dims(X[0], axis=0)

In [140]:
model.predict_classes(test)

array([300])

In [103]:
model.predict_classes(seed_text)

ValueError: Error when checking : expected embedding_5_input to have shape (25,) but got array with shape (1,)

In [85]:
np.full(25, 2)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2])

In [154]:
sequences[2018]

array([ 22,  67,   0,   2,  11,   1, 225,  25, 589, 335,   0,  45,   5,
        93, 119,  12,  49, 244, 188,   0,  13,  28,   3, 412, 156,   0])

In [164]:
def generate_songs(model, sequences, seq_length, reverse_dictionary, n_words=1000, seed_num=2018
#                    generate_random=True
                  ):
    seed_text = sequences[seed_num]
#     seed_text = gen_seed(sequences=sequences)[:seq_length]
    start = [reverse_dictionary[index] for index in seed_text]
    print('start: ', ' '.join(start))
    
    result = []
    
    for _ in range(n_words):
        in_text = np.expand_dims(seed_text, axis=0)
        yhat = model.predict_classes(in_text, verbose=0)
        if yhat == 2500:
            word = words.words()[np.random.randint(0,len(words.words()))]
        else:
            word = reverse_dictionary[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-seq_length:]
        result.append(word)
            
    return ' '.join(result)

In [143]:
generate_songs(
    model=model,
    sequences=X,
    seq_length=25,
    reverse_dictionary=index_to_word,
    n_words=1000, 
    file_name='the_decemberists.txt'
)

start:  postman do you have a letter for me eol a letter for me eol from my own true love eol lost at sea eol lost


'at sea eol mr postman do you have a letter for me eol mr postman do you have a letter for me eol chorus eol and try one and try two eol guess it always comes down to eol alright its okay eol guess it always comes down in wait for symmetry eol and were was a county lineman eol on a high line on his arm eol arms but i take us jimmy row eol oh never begin eol i will hang my love my love eol my bones fill unknown eol go to shoulder eol youll not betray us to me eol clawing at the ceiling eol of my grave eol i am the only writer off the jewess and the brow eol i have unknown that i am you here eol didnt i didnt i didnt i eol my life dont see thats unknown a unknown drowned eol over you eol but something went limp beneath the wind eol and all the waves will be here eol wont quell my want for love eol and i may swoon from all this swelling eol but i wont want for love eol william colin meloy eol oh my own true love eol two away unknown in unknown eol go down down eol go by dirigible eol well

In [148]:
generate_songs(
    model=model,
    sequences=X,
    seq_length=25,
    reverse_dictionary=index_to_word,
    n_words=1000, 
    file_name='the_decemberists.txt'
)

start:  rolls down the drain eol oh what a lonely thing eol in a lonely drain eol chorus eol july july july eol it never seemed


'so strange eol july july july eol it never seemed so eol it never seemed so strange eol fifteen picking on your weeping bother eol the lash unknown fifteen shins eol you you wont change you down eol and the sun breaks to no youre a common collective eol i am imprinted upon your stars eol and raise a glass to unknown of the season eol witness to the old main drag eol down by the old main drag eol down by the water and eol down her got shaking eol in a bloodlines eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go eol and if you dont love me let me go e

In [149]:
generate_songs(
    model=model,
    sequences=X,
    seq_length=25,
    reverse_dictionary=index_to_word,
    n_words=1000, 
    file_name='the_decemberists.txt'
)

start:  so you bend back and shake at the frame eol of the frame you made eol but dont you shake alone eol please avery come


'home eol head strong you and let me see eol its calm a legs were between hard eol a soft on the heads eol and youre a common way to go eol get to all too slow eol but ill shake the uncle daughter eol chorus eol and then she came here and calm and easy and willow me unknown eol hey hey hey as the old main lioness eol and we hold in our sigh eol dont carry it all dont carry it all eol hes keep on stealin eol if you for you and you were made for me eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it to ten you wont make it again eol and if you ever make it

In [9]:
model_100 = models.load_model('model_checkpoints/weights-improvement-100-0.5531.hdf5')
model_75  = models.load_model('model_checkpoints/weights-improvement-75-0.9800.hdf5')
model_50  = models.load_model('model_checkpoints/weights-improvement-50-1.6432.hdf5')
model_25  = models.load_model('model_checkpoints/weights-improvement-25-2.8392.hdf5')
model_1   = models.load_model('model_checkpoints/weights-improvement-01-5.8760.hdf5')

In [159]:
model_names = ['model_1', 'model_25', 'model_50', 'model_75', 'model_100']
for count, model in enumerate([model_1, model_25, model_50, model_75, model_100]):
    name = model_names[count]
    generated_lyrics = generate_songs(
                            model=model,
                            sequences=X,
                            seq_length=25,
                            reverse_dictionary=index_to_word,
                            n_words=100, 
                            file_name='the_decemberists.txt'
                        )
    generated_lyrics = generated_lyrics.replace(' eol ', ' \n ').replace(' eos ', ' \n\n\n ')
    
    with open
    
    
    
    print(name + ':\n', generated_lyrics)

start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
model_1:
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol 
 eol eol
start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
model_25:
 full 
 gazing and the lulling of the cafe bars 
 the bullets of the perfect crime 
 chorus 
 and i will hang my head hang my head hang my head hang my head hang my head low 
 and i dont swoon from build your wall 
 hold to make your precious collective 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its foregone 
 its 

In [166]:
model_names = ['model_1', 'model_25', 'model_50', 'model_75', 'model_100']
for count, model in enumerate([model_1, model_25, model_50, model_75, model_100]):
    name = model_names[count]
    generated_lyrics = generate_songs(
                            model=model,
                            sequences=X,
                            seq_length=25,
                            reverse_dictionary=index_to_word,
                            n_words=100
                        )
    
    generated_lyrics = generated_lyrics.replace(' eol ', ' \n ').replace(' eos ', ' \n\n\n ')
    
    file_name='the_decemberists_seed_num2018.txt'
    
    with open(file_name, 'a+') as f:
        print(name + ':\n', generated_lyrics + '\n\n\n', file=f)

start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane
start:  so long eol and all the stars were crashing round eol as i laid eyes on what id found eol it was a white crane


In [17]:
def input_cleaner(text_in):
    text_in = text_in.lower().replace(' n ', ' eol ').replace('[verse ', '[verse')
    text_in = text_in.replace("'", '').replace('-', ' ')
    tokens = text_in.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    return tokens



def generate_seed_text_from_input():
    
    input_text = input('Type what you want here.')
    
    seed_text = input_cleaner(input_text)
    
    seed_text = [word_to_index[word] if word in word_to_index else word_to_index['unknown'] for word in seed_text]
    
    seed_text = pad_sequences([seed_text], maxlen=25, value=2500)
    
    print('Rendering text as lyrics...')
    
    return seed_text, input_text



def generate_song_from_input(model, seq_length, reverse_dictionary, n_words, seed_text):
    
    print('Writing new song')
    
    result = []
    
    seed_text = np.squeeze(seed_text, axis=0)
    
    for _ in range(n_words):
        in_text = np.expand_dims(seed_text, axis=0)
        yhat = model.predict_classes(in_text)
        if yhat == 2500:
            word = words.words()[np.random.randint(0,len(words.words()))]
        else:
            word = reverse_dictionary[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-seq_length:]
        result.append(word)
           
    return ' '.join(result)

def generate_new_song():
    model = model_50
    seq_length = 25
    reverse_dictionary = index_to_word
    n_words = int(input('How many words?'))
    
    seed_text, input_text = generate_seed_text_from_input()
    
    lyrics = generate_song_from_input(model, seq_length, reverse_dictionary, n_words, seed_text)
    
    lyrics = lyrics.replace(' eol ', ' \n ').replace(' eos ', ' \n\n\n ')
    
    complete_song = input_text + '\n' + lyrics
    
    print('New song finished!')
    
    return complete_song

In [21]:
new_song = generate_new_song()

How many words?500
Type what you want here.yonder mountains knew the truth but could not speak
Rendering text as lyrics...
Writing new song
New song finished!


In [22]:
print(new_song)

yonder mountains knew the truth but could not speak
that superaccessory and hummocky 
 lost out hammering 
 parapegma taking compress in white and is wait for me 
 Rabbinic and sunfisher and puriri and rufflike and tain and vervel and repine and scream and willow me in stardom 
 because the slubby of the snavel and walls in the Xiphias 
 and the wanting comes in waves 
 and the ricks and the racks and the run 
 and i am a brawn 
 i got the cavalry captain 
 i am the remedy to the Turkophobist 
 and if you dont love me let me go 
 and if you dont love me let me go 
 thatll teach him 
 columbine columbine 
 the pearliness of the rollicksome of the circle k 
 so the wanting comes in waves 
 thatll teach him 
 and a queen all over themselves 
 we laid edgar styrofoam 
 to a lone concertina they drank in tow 
 in the water of the seine 
 it’ll eat a levigate wall and quiet 
 like a heads 
 among the bombs of broth wall where the infiniteness of the freeway 
 and the wanting comes in waves 


In [36]:
test = load_data(filename='all_songs.csv', col='is_folk', col_value=1)

In [37]:
test.shape

(5365,)

In [32]:
df = pd.read_csv('all_songs.csv')

In [35]:
df.loc[df['is_folk'] == 1]

(5365, 18)

In [168]:
seed_text, input_text = generate_seed_text_from_input()

In [171]:
new_song = generate_song_from_input(model_75, 25, reverse_dictionary=index_to_word, n_words=100, seed_text=seed_text)

'eol april more and iliococcygeal and interstriation and scream and willow eos from eos eol its a shallow little frame eol and its a unbacked to the paronomasian eol and i am a brawn eol i got the cavalry captain eol i am a brawn eol i am a writer of a politicalize irradiate eol and the ricks and the racks and the run eol and i am a brawn eol i got the cavalry captain eol i am the remedy to the dispossess eol and if you dont with eos let eos go eol and if you dont with'